# 📄 生成 AT 指令文档（含 自动 conf.py + 参数json 支持）

In [ ]:
!pip install pandas jinja2 sphinx sphinx-rtd-theme

In [ ]:
import os
import pandas as pd
import json
from jinja2 import Environment
from collections import defaultdict

In [ ]:
# ✅ 如果 docs/source/conf.py 不存在，则自动创建它
conf_path = "docs/source/conf.py"
if not os.path.exists(conf_path):
    os.makedirs(os.path.dirname(conf_path), exist_ok=True)
    with open(conf_path, "w", encoding="utf-8") as f:
        f.write("""
import os
import sys
sys.path.insert(0, os.path.abspath('.'))

project = 'AT Command Manual'
author = 'Your Name'
release = '1.0'

extensions = []
templates_path = ['_templates']
exclude_patterns = []

html_theme = 'sphinx_rtd_theme'
html_static_path = ['_static']

def setup(app):
    app.add_css_file('custom.css')
""")
    print("✅ conf.py 已创建")
else:
    print("ℹ️ conf.py 已存在")

# 创建 custom.css in static
static_dir = 'docs/source/_static'
os.makedirs(static_dir, exist_ok=True)
with open(os.path.join(static_dir, 'custom.css'), 'w', encoding='utf-8') as f:
    f.write("body { font-family: 'Microsoft YaHei', sans-serif; }")
print("✅ CSS 静态文件准备完成")

In [ ]:
# 读取 CSV 并准备模板
CSV_PATH = 'at_commands_with_json.csv'  # 你的 CSV 路径
OUTPUT_DIR = 'docs/source'

TEMPLATE_STRING = """
.. _cmd-{{ command_name|lower }}:

{{ command_name }} — {{ command_title }}
{{ '=' * command_name|length }}

{{ description or '' }}

执行命令
^^^^^^^^

**命令：**

::

    {{ command }}

**响应：**

::

{% for line in response.splitlines() %}    {{ line }}
{% endfor %}

参数
^^^^
{% if parameters %}
{% for name, data in parameters.items() %}
- **{{ name }}**：
  {% if data.get('__desc__') %}
    {{ data.__desc__ }}
  {% endif %}
  {% for key, value in data.items() if key != '__desc__' %}
    - {{ key }}：{{ value }}
  {% endfor %}
{% endfor %}
{% else %}
无
{% endif %}

说明
^^^^
{{ note or '无' }}

示例命令
^^^^^^^^

::

{% for line in example.splitlines() %}    {{ line }}
{% endfor %}
"""

In [ ]:
df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip()
env = Environment()
template = env.from_string(TEMPLATE_STRING)

chapters = df.groupby('章节')
chapter_commands = defaultdict(list)
chapter_names = []

for chap, grp in chapters:
    chap_name = str(chap).strip()
    chap_dir = os.path.join(OUTPUT_DIR, chap_name)
    os.makedirs(chap_dir, exist_ok=True)
    chapter_names.append(chap_name)

    for _, row in grp.iterrows():
        cmd = str(row['命令']).strip()
        out_file = os.path.join(chap_dir, f"{cmd}.rst")

        try:
            parameters = json.loads(row.get('参数json', '{}'))
        except Exception:
            parameters = {}

        content = template.render(
            command_name=cmd,
            command_title=str(row.get('命令标题', '')).strip(),
            command_type=str(row.get('命令类型', '')).strip(),
            command=str(row.get('命令格式', '')).strip(),
            response=str(row.get('响应', '')).strip(),
            description=str(row.get('功能描述', '')).strip(),
            note=str(row.get('备注', '')).strip(),
            parameters=parameters,
            example=str(row.get('示例命令', '')).strip()
        )

        with open(out_file, 'w', encoding='utf-8') as f:
            f.write(content)

        chapter_commands[chap_name].append(cmd)

print("✅ 所有 RST 已生成")

In [ ]:
from jinja2 import Template

chapter_tmpl = """{{ chapter }}
{{ '=' * chapter|length }}

.. toctree::
   :maxdepth: 1

{% for c in cmds %}   {{ c }}
{% endfor %}
"""

main_tmpl = """AT 指令文档
===============

.. toctree::
   :maxdepth: 1
   :caption: 章节目录

{% for ch in chapter_names %}   {{ ch }}/index
{% endfor %}
"""

for chap, cmds in chapter_commands.items():
    path = os.path.join(OUTPUT_DIR, chap, 'index.rst')
    with open(path, 'w', encoding='utf-8') as f:
        f.write(Template(chapter_tmpl).render(chapter=chap, cmds=cmds))

main_path = os.path.join(OUTPUT_DIR, 'index.rst')
with open(main_path, 'w', encoding='utf-8') as f:
    f.write(Template(main_tmpl).render(chapter_names=chapter_names))

print("✅ index.rst 完成")

In [ ]:
!sphinx-build -b html docs/source docs/build/html -c docs/source
print("✅ HTML 构建完成，可查看 docs/build/html/index.html")